This code generates the model SNe for each model type and prints the fraction of emission in HII regions for the controls in the paper.


In [1]:
#import global packages
import time
import os
import math
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
import matplotlib as mpl
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.table import Table, join
from reproject import reproject_interp
import aplpy
import scipy
from scipy.interpolate import interp1d
import seaborn as sns
from astropy.coordinates import SkyCoord, Angle
from astropy.nddata import Cutout2D

# import local packages
import sys
sys.path.append('./Python')
from SNeGenerationModels import runModels
from FindNearestHIIRegion import nearestHII

# map extensions
ext = "HA_FLUX_CORR"
hii_ext = 0


In [2]:
# Compile galaxy and sne data
data = Table.read('../Data/1.GroupedSampleData.csv', format='csv')

data[0:1]

col0,galaxy,distance,gal_ra,gal_dec,pos_ang,incl,nat_res,r_eff,maps_musedap,maps_nat,maps_150,maps_1kpc,maps_HII,maps_irac,maps_CO21,maps_eCO21,maps_ext,maps_vdisp,maps_HST,JWST200,JWST300,JWST335,JWST360,JWST770,JWST1000,JWST1130,JWST2100,sn_name,sn_type,sn_ra,sn_dec
int64,str7,float64,float64,float64,float64,float64,float64,float64,str71,str58,str64,str63,str50,str40,str72,str73,str36,str32,str45,str93,str93,str93,str93,str93,str94,str94,str94,str82,str50,str127,str132
0,NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,42.091877,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_1kpc.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,../Data/SFR_v1/NGC1087_MAPS_sfr.fits,../Data/vdisp/ngc1087_vdisp.fits,../../GalaxyFiles/HST/ngc1087_hst_ha_sic.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F200W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F300W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F330W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F365W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F770W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F1000W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F1130W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F2100W_atGauss7p5_anchored.fits,['SN1995V'],['II'],[41.61152777777777],[-0.4987861111111111]


In [3]:
# prepare lists for running models

ccs = [(float(data["gal_ra"][i]), float(data["gal_dec"][i])) for i in range(len(data))] 
    
sn_ras, sn_decs, sn_names = [],[],[]
for i in range(len(data)):
    sn_ras.append(data["sn_ra"][i][1:-1].replace(",","").split(" "))
    sn_decs.append(data["sn_dec"][i][1:-1].replace(",","").split(" "))
    sn_names.append(data["sn_name"][i][1:-1].replace(",","").replace("'","").split(" "))

for i in range(len(sn_ras)):
    sn_ras[i] = [float(ele) for ele in sn_ras[i]]
    sn_decs[i] = [float(ele) for ele in sn_decs[i]]
    
sn_name = np.hstack(sn_names)

In [4]:
# this cell takes a few hours to run. Don't uncomment and run unless necessary!
# run models 1-4 and store results
galCol, counter, int_mod_rand5, int_mod_rand1, int_mod_gas, int_mod_star = [],[],[],[],[],[]
near_mod_r5, near_mod_r1, near_mod_gs, near_mod_st = [],[],[],[]
iran5,iran1,igas,istar,nran1,nran5,ngas,nstar = [],[],[],[],[],[],[],[]
r5ras,r5decs,r1ras,r1decs = [],[],[],[]

for i in range(len(data["galaxy"])):
# for i in range(0,2):    
    # Using MUSE map for coverage and to measure H-alpha emission
    Ha_hdu = pyfits.open(data["maps_nat"][i])
    Ha_map = Ha_hdu[ext].data  

    #Convert x & y pixels to ra and dec
    wcs      = WCS(Ha_hdu[ext].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)
            
    #flatten data structures 
    f_Ha     = Ha_map.flatten()    
    f_ra     = ra.flatten()
    f_dec    = dec.flatten()    
    
    #Get IRAC maps to look at the stellar populations emission
    SD_hdu = pyfits.open(data["maps_irac"][i])
    # Reproject IRAC maps
    SD_repro, SD_foot = reproject_interp(SD_hdu, Ha_hdu[ext].header)
    f_SD   = SD_repro.flatten() 

    #remove nans for Halpha and IRAC overlap
    keep  = np.where(np.isfinite(f_SD) & np.isfinite(f_Ha))
    IRAC     = f_SD[keep]
    star_ra  = f_ra[keep]
    star_dec = f_dec[keep]
    
    print("On Galaxy: ", i+1, "of ", len(data["galaxy"]), data["galaxy"][i])
    
    # run individual models for each local SN site
    for j in range(len(sn_ras[i])):
        print("On SN", j+1, "of", len(sn_ras[i]))
        print("On Model 1 of 8")
        r5ra, r5dec, r5dx, r5dy, intRand500  = runModels(data["galaxy"][i], data["maps_nat"][i], ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], sn_ras[i][j], sn_decs[i][j], modelType=1, local=500)
        print("On Model 2 of 8")
        r1ra, r1dec, r1dx, r1dy, intRand1000 = runModels(data["galaxy"][i], data["maps_nat"][i], ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], sn_ras[i][j], sn_decs[i][j], modelType=1, local=1000)
        print("On Model 3 of 8")
        gsra, gsdec, gsdx, gsdy, intGas      = runModels(data["galaxy"][i], data["maps_nat"][i], ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], sn_ras[i][j], sn_decs[i][j], modelType=2)
        print("On Model 4 of 8")
        stra, stdec, stdx, stdy, intStar     = runModels(data["galaxy"][i], data["maps_nat"][i], ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], sn_ras[i][j], sn_decs[i][j], modelType=3, starLight = IRAC, starRa = star_ra, starDec = star_dec)    
        print("On Model 5 of 8")
        n_mod_r5_in_hii, n_mod_r5_dist, n_mod_r5_ra, n_mod_r5_dec, n_mod_r5_hii_val = nearestHII(data["galaxy"][i], data["maps_HII"][i], hii_ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], r5ra, r5dec)
        print("On Model 6 of 8")
        n_mod_r1_in_hii, n_mod_r1_dist, n_mod_r1_ra, n_mod_r1_dec, n_mod_r1_hii_val = nearestHII(data["galaxy"][i], data["maps_HII"][i], hii_ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], r1ra, r1dec)
        print("On Model 7 of 8")
        n_mod_gs_in_hii, n_mod_gs_dist, n_mod_gs_ra, n_mod_gs_dec, n_mod_gs_hii_val = nearestHII(data["galaxy"][i], data["maps_HII"][i], hii_ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], gsra, gsdec)
        print("On Model 8 of 8")
        nn_mod_st_in_hii, n_mod_st_dist, n_mod_st_ra, n_mod_st_dec, n_mod_st_hii_val = nearestHII(data["galaxy"][i], data["maps_HII"][i], hii_ext, ccs[i], data["pos_ang"][i], data["incl"][i], data["distance"][i], stra, stdec)

        iran5.append(intRand500)
        iran1.append(intRand1000)
        igas.append(intGas)
        istar.append(intStar)
        nran5.append(np.hstack(n_mod_r5_dist))
        nran1.append(np.hstack(n_mod_r1_dist))
        ngas.append(np.hstack(n_mod_gs_dist))
        nstar.append(np.hstack(n_mod_st_dist)) 
        r5ras.append(r5ra)
        r5decs.append(r5dec)
        r1ras.append(r1ra)
        r1decs.append(r1dec)
    
    print("---------------------------------------")
    
print("Combining arrays:")

# near_mod_r5 = np.hstack(nran5[0])
# near_mod_r1 = np.hstack(nran1[0])    
# near_mod_gs = np.hstack(ngas[0])    
# near_mod_st = np.hstack(nstar[0])    
# int_mod_rand5 = np.hstack(iran5[0])
# int_mod_rand1 = np.hstack(iran1[0])
# int_mod_gas   = np.hstack(igas[0])
# int_mod_star  = np.hstack(istar[0])

# near_mod_r5 = np.hstack(nran5)
# near_mod_r1 = np.hstack(nran1)    
# near_mod_gs = np.hstack(ngas)    
# near_mod_st = np.hstack(nstar)    
# int_mod_rand5 = np.hstack(iran5)
# int_mod_rand1 = np.hstack(iran1)
# int_mod_gas   = np.hstack(igas)
# int_mod_star  = np.hstack(istar)
# r_5_ras  = np.hstack(r5ras)
# r_5_decs = np.hstack(r5decs)
# r_1_ras  = np.hstack(r1ras)
# r_1_decs = np.hstack(r1decs)



        
        
    

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
On Galaxy:  1 of  10

On Galaxy:  7 of  10 NGC3627
On SN 1 of 5
On Model 1 of 8
On Model 2 of 8
On Model 3 of 8
On Model 4 of 8
On Model 5 of 8
On Model 6 of 8
On Model 7 of 8
On Model 8 of 8
On SN 2 of 5
On Model 1 of 8
On Model 2 of 8
On Model 3 of 8
On Model 4 of 8
On Model 5 of 8
On Model 6 of 8
On Model 7 of 8
On Model 8 of 8
On SN 3 of 5
On Model 1 of 8
On Model 2 of 8
On Model 3 of 8
On Model 4 of 8
On Model 5 of 8
On Model 6 of 8
On Model 7 of 8
On Model 8 of 8
On SN 4 of 5
On Model 1 of 8
On Model 2 of 8
On Model 3 of 8
On Model 4 of 8
On Model 5 of 8
On Model 6 of 8
On Model 7 of 8
On Model 8 of 8
On SN 5 of 5
On Model 1 of 8
On Model 2 of 8
On Model 3 of 8
On Model 4 of 8
On Model 5 of 8
On Model 6 of 8
On Model 7 of 8
On Model 8 of 8
---------------------------------------
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs

In [5]:
np.hstack(n_mod_r5_dist)

array([ 36.98059062,   0.        , 107.33822659, 267.64233899,
       151.91970683, 131.51638816, 246.18397041,  67.55709278,
       115.66183763,  89.72171094,  65.10850014,  25.96671008,
        86.41755519,  30.97838297, 135.11420624, 108.25062543,
       214.32059727, 127.78326948,   0.        , 103.86669071,
       137.60366356,  61.35845978, 107.99990987, 210.21447903,
       121.96016409,   0.        , 218.66090329,  89.72173563,
        18.24077225,  72.96323864,  36.48161779,   0.        ,
        21.70284051,  15.4891951 ,  77.26597572, 127.78335188,
       128.06076631,   0.        ,   0.        , 126.35931241,
        30.97841   , 303.23331398, 215.99981385, 187.06495553,
        33.00200458, 235.80918069, 109.44450121, 170.38112649,
       213.89150161,  89.72117789, 109.44491972,   0.        ,
         0.        , 224.77264296,  53.66912642, 198.89004111,
        77.26597572,  18.24082933, 119.11877607, 144.96394143,
        25.96667985, 105.77023276,  95.21637394,  54.12

In [6]:
near_mod_r5= np.hstack(nran5)
near_mod_r1 = np.hstack(nran1)    
near_mod_gs = np.hstack(ngas)    
near_mod_st = np.hstack(nstar)    
int_mod_rand5 = np.hstack(iran5)
int_mod_rand1 = np.hstack(iran1)
int_mod_gas   = np.hstack(igas)
int_mod_star  = np.hstack(istar)
r_5_ras  = np.hstack(r5ras)
r_5_decs = np.hstack(r5decs)
r_1_ras  = np.hstack(r1ras)
r_1_decs = np.hstack(r1decs)



results = Table({"int_mod_rand5": int_mod_rand5,
                "int_mod_rand1": int_mod_rand1,
                "int_mod_gas": int_mod_gas,
                "int_mod_star": int_mod_star,
                "near_mod_rand5": near_mod_r5,
                "near_mod_rand1": near_mod_r1,
                "near_mod_gas": near_mod_gs,
                "near_mod_star": near_mod_st,
                "r_5_ras": r_5_ras,
                "r_5_decs": r_5_decs,
                "r_1_ras": r_1_ras,
                "r_1_decs": r_1_decs})

results.write("../Data/1.model_sne_ints_dists.csv", format = "csv", overwrite = True)

print("finished")

finished


In [7]:
print(len(nran5))

33


In [8]:
print(len(iran5[0]))

1000
